# Expérimentations - GCN for text classification - Dataset R8

Dans ce notebook, nous allons tenter de recoder l'algorithme décrit dans le papier "GCN for text classification", et d'effectuer des expériences autour de celui-ci.

Notre objectif sera de classifier correctement des textes, en n'ayant qu'une petite base d'entrainement.

A partir d'un corpus de textes, dont seulement un petit pourcentage est labelisé, nous soihaitons attribuer des classes à tous les documents. Le principe sera de répandre les classes des textes labelisés grâce à un GCN. Pour cela, nous devrons donc représenter nos données sous forme d'un graphe.

Nous allons donc construire un graphe hétérogène dont les sommets seront des mots (apparaisant dans les documents) et des documents. Les arêtes représenteront des liens entre des mots, ou bien entre mots et documents, mais il n'y a pas d'arêtes entre documents. 

Le graphe contiendra tous les documents du corpus, c'est à dire autant le test set que le train set. Néanmoins, seul un petit pourcentage de ces documents disposerons de labels.

Une fois le graphe construit nous pourrons lui appliquer un Graph Convolutional Network, dans le but de classifier les documents non labelisés.

## EXPLIQUER ICI FONCTIONNEMENT DU GCN

Nous pourrons donc mesurer les performances de notre modèle en regardant le taux de documents bien classifiés (parmi les documents qui ne disposaient pas de labels avant le GCN bien sur).

Nous pourrons enfin tenter diverses expérimentations, notamment :

    - Essayer plusieurs pourcentages de données labelisées
    - embedding à la place  de pmi (developper)
    - ...

### Table des matières <a name="up"></a>

* [1 Import des textes](#import)
* [2 Préparation des données](#init)
* [3 Calcul du TFIDF et du PMI](#dico)
* [4 Construction du graphe](#tfidf)
* [5 Text Rank](#tr)
* [6 affichage docx](#docx)
* [7 keywords](#keywords)

Nous commençons par importer les librairies dont nous aurons besoin.

Parmi celles-ci, plusieurs librairies de NLP, qui nous servirons notamment pour le pré-processing des données, et la tokenization (c'est à dire pour séparer les documents en mots). Nous aurons également besoin de la librairie networkx qui permet de gérer les graphes.


In [157]:
#math
import numpy as np
import math
import random

#NLP
import gensim
from gensim.models import TfidfModel
from gensim import matutils
from convectors.layers import (Contract, Lemmatize, Phrase, TfIdf,Tokenize)

#Affichage du temps et accès aux données
from tqdm import tqdm
import os

#Structure de données
from scipy.sparse import csc_matrix, lil_matrix, csr_matrix

#Graphes
import networkx as nx
import networkx.algorithms.community as nx_comm
from textgraph.graph import TextGraph

# 1 Import des textes <a name="import"></a>

[Retour table des matières](#up)

Pour importer les textes de notre dataset, nous avons décidé de reprendre et adapter le code des auteurs du papier, afin d'être certains d'avoir exactement les mêmes données et ainsi pouvoir comparer nos résultats.


Nous créons ainsi plusieurs listes.

    - doc_name_list est la liste de tous les triplets (id, test/train, label) en strings.
    - doc_train_list et doc_test_list contiennent, sous la même forme que la liste précédente, les triplets des test et train set respectivement.
    - doc_content_list est la liste des textes.
    - train_ids est la liste des id d'entrainement.
    - train_ids_str est la même chose, mais sous forme de string avec les ids séparés par des \n
    - shuffle_doc_name_list correspond à doc_name_list, mais mélangé aléatoirement.
    - shuffle_doc_words_list correspond à doc_content_list mais mélangé aléatoirement.
    
Cette partie du code (import des données) est fortement inspirée du [github des auteurs](https://github.com/yao8839836/text_gcn).

In [158]:
dataset = 'R8'

In [159]:
# shuffling
doc_name_list = []
doc_train_list = []
doc_test_list = []

f = open(os.path.join('text_gcn','data', 'R8.txt'), 'r')
lines = f.readlines()
for line in lines:
    doc_name_list.append(line.strip())
    temp = line.split("\t")
    if temp[1].find('test') != -1:
        doc_test_list.append(line.strip())
    elif temp[1].find('train') != -1:
        doc_train_list.append(line.strip())
f.close()

In [160]:
doc_content_list = []
f = open('text_gcn/data/corpus/' + dataset + '.clean.txt', 'r')
lines = f.readlines()
for line in lines:
    doc_content_list.append(line.strip())
f.close()
# print(doc_content_list)

train_ids = []
for train_name in doc_train_list:
    train_id = doc_name_list.index(train_name)
    train_ids.append(train_id)
#print(train_ids)
random.shuffle(train_ids)

# partial labeled data
#train_ids = train_ids[:int(0.2 * len(train_ids))]

train_ids_str = '\n'.join(str(index) for index in train_ids)
f = open('text_gcn/data/' + dataset + '.train.index', 'w')
f.write(train_ids_str)
f.close()

test_ids = []
for test_name in doc_test_list:
    test_id = doc_name_list.index(test_name)
    test_ids.append(test_id)
#print(test_ids)
random.shuffle(test_ids)

test_ids_str = '\n'.join(str(index) for index in test_ids)
f = open('text_gcn/data/' + dataset + '.test.index', 'w')
f.write(test_ids_str)
f.close()

ids = train_ids + test_ids
#print(ids)
#print(len(ids))

shuffle_doc_name_list = []
shuffle_doc_words_list = []
for id in ids:
    shuffle_doc_name_list.append(doc_name_list[int(id)])
    shuffle_doc_words_list.append(doc_content_list[int(id)])
shuffle_doc_name_str = '\n'.join(shuffle_doc_name_list)
shuffle_doc_words_str = '\n'.join(shuffle_doc_words_list)

f = open('text_gcn/data/' + dataset + '_shuffle.txt', 'w')
f.write(shuffle_doc_name_str)
f.close()

f = open('text_gcn/data/corpus/' + dataset + '_shuffle.txt', 'w')
f.write(shuffle_doc_words_str)
f.close()

# 2 Préparation des données <a name="init"></a>

[Retour table des matières](#up)

Maintenant que nous avons importé les données sous les formes qui nous conviennent, nous devons les préparer.

Nos données sont déjà partiellement préparées, puisque la ponctuation et les caractères spéciaux ont été retirés, et les textes mis en minuscules. 

En particulier, nous tokenizons les données, c'est à dire que nous séparons chaque document en liste de mots. Parmi ces mots, nous retirons les stopwords, c'est à dire les mots qui apparaissent très souvent et ne porte pas beaucoup de sens. C'est le cas notamment des pronoms, mais aussi de certains verbes comme être, avoir, etc. Il existe des listes de stopwords préfaites. En l'occurence nous utilisons une librairie qui elle même utilise une liste de stopwords issue de la librairie NLTK (spécialisée dans le NLP). Enfin, nous lemmatisons les mots. C'est à dire que nous les ramenons à leur racine. Par exemple, "thinking" et 'thoughts" deviendront "think".

Pour finir, nous retirons les mots qui apparaissent dans plus de 20% des documents. En effet, nous pouvons considérer qu'un mot qui apparait dans plus de 1 document sur 5 (parmi plusieurs milliers de textes) n'apporte pas beaucoup de sens. Il s'agit de stopwords également.

In [161]:
def prep(data, no_below=1, no_above=0.2, max_words=100000):
    
    word_tok = Contract()
    word_tok += Tokenize(stopwords=["en","fr"])
    word_tok += Lemmatize()
    word_tok += Phrase()
    word_tok.verbose=False
    #word_tok += Snowball(lang="french") #stemming
    
    tokens=word_tok(data)
    #print((tokens)) = liste des mots par doc
    
    dictionary = gensim.corpora.Dictionary(tokens)
    #print(dictionary)
    
    #doesn't keep words that appear in less than no_below docs or more than no_above (ratio) docs
    dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=max_words)
    #print(dictionary) 
    
    return word_tok, dictionary

In [162]:
word_tok, dictionary = prep(doc_content_list, 0.2)

In [163]:
dictionary[50]

'improvement'

In [164]:
vocab[50]

'accruing'

# 3 Calcul du TFIDF et du PMI<a name="dico"></a>

[Retour table des matières](#up)

Nous calculons maintenant le score TFIDF, qui nous servira pour les poids des arêtes entres les liens et documents.

Le score TFIDF traduit l'idée qu'un mot qui apparait beaucoup dans un document mais assez peu dans le corpus est probablement un mot important relativement au document. Un mot qui apparait très peu dans un texte est assez logiquement peu important (sauf s'il n'apparait presque jamais dans le reste du corpus !). Enfin un mot qui apparait souvent dans le document mais qui apparait également souvent dans le corpus n'est pas très important. Il s'agit également d'une forme de stopwords.

Le score TFIDF est donc une mesure statistique basée sur la fréquence des mots.

Il se calcule selon la formule qui suit :

## INSERER FORMULE TFIDF

In [195]:
def compute_tfidf(data, dictionary, word_tok, option=1):
    
    tokenized_data = word_tok(data)
    
    tokenized_data_idx = []
    for i in range(len(data)):
        tokenized_data_idx.append(dictionary.doc2idx(tokenized_data[i]))
    
    if option : #1
        corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]
        tfidf = TfidfModel(corpus) #fit
        X = tfidf[corpus] #transform
        TFIDF = matutils.corpus2csc(X,num_docs=len(X)).T
    
        vocab = list(dictionary.token2id.keys())
        
    else: #0
        vectorizer = TfIdf(
            min_df=1,
            max_df=.5,
            max_features=None,
            verbose=False)

        TFIDF = vectorizer(tokenized_data)
        vocab = vectorizer.vectorizer.get_feature_names_out()
        
    #Version dense et datafrmame
    #mat_dense = TFIDF.toarray()
    #df_tfidf = pd.DataFrame(mat_dense, columns=vocab)    
    
    return TFIDF, tokenized_data, tokenized_data_idx, vocab

In [233]:
tokenized_data_idx[0]

[8,
 15,
 3,
 20,
 19,
 8,
 15,
 -1,
 -1,
 6,
 10,
 2,
 21,
 14,
 20,
 19,
 9,
 -1,
 18,
 17,
 4,
 -1,
 0,
 -1,
 6,
 22,
 16,
 18,
 1,
 13,
 4,
 12,
 5,
 7,
 20,
 11,
 -1,
 -1,
 -1,
 -1]

In [197]:
TFIDF, tokenized_data, tokenized_data_idx, vocab = compute_tfidf(doc_content_list, dictionary, word_tok, 0)

In [231]:
dictionary[15]

'product'

In [199]:
tokenized_data

0       [champion, product, approves, stock, split, ch...
1       [computer, terminal, system, complete, sale, c...
2       [inc, year, net, shr, ct, vs, dlrs, net, vs, a...
3       [international, inc, nd, qtr, jan, oper, shr, ...
4       [brown, forman, inc, th, qtr, net, shr, one, d...
                              ...                        
7669    [balladur, maintenance, louvre, accord, french...
7670    [philippin, trade, gap, widens, january, augus...
7671    [iran, soviet, union, swap, crud, refined, pro...
7672    [chase, corp, make, offer, entregrowth, chase,...
7673    [japan, india, conference, cut, gulf, war, ris...
Length: 7674, dtype: object

Nous calculons désormais le PMI (Pointwise Mutual Information). Cela nous servira pour les poids des arêtes entre deux mots. Le PMI mesure à quel point 2 mots sont susceptibles d'apparaitre ensemble. Il s'agit donc d'une mesure de la co-occurence. L'hypothèse faite est que deux mots qui apparaissent souvent ensemble sont susceptibles d'avoir un sens proche.

Pour calculer le PMI, il faut faire glisser une fenêtre sur chacun des textes. La taille de cette fenêtre est un hyper-paramètre à choisir.

En parcourant les documents, nous compterons : 
    - Le nombre total de fenêtres : #W
    - Le nombre de fenêtres qui contiennent le mot i, pour chaque mot i : #W(i)
    - Le nombre de fenêtres qui contiennent 2 mots i et j : #W(i,j)
    
Finalement, le PMI se calcule comme suit : 

# INSERER FORMULE

In [200]:
def ordered_word_pair(a, b):
    if a > b:
        return b, a
    else:
        return a, b
        
def update_word_and_word_pair_occurrence(q, W_i, W_ij):
    '''
    Traitement des mots de une fenêtre
    
    q : set d'indices de mots
    '''
    #On utilise un set pour avoir chaque mot une seule fois
    #L'ordre n'est pas important
    unique_q = list(set(q))
    #On retire les mots qui n'apparaissent pas dans le dictionnaire
    #Cad les mots dont la valeur est -1
    if -1 in unique_q:
        unique_q.remove(-1)
    for w in unique_q:
        #Update du nb de fentres ou apparait chaque mot
        try:
            W_i[w] += 1
        except:
            W_i[w] = 1 #Si la valeur n'existe pas encore
    for i in range(len(unique_q)):
        #update du nb de fenetres ou apparait chaque paire
        for j in range(i+1, len(unique_q)):
            #Pour chaque mot j après i 
            word1 = unique_q[i]
            word2 = unique_q[j]
            #On ne veut update que un coté de la matrice
            word1, word2 = ordered_word_pair(word1, word2)
            try:
                W_ij[word1, word2] += 1
            except:
                W_ij[word1, word2] = 1
    return W_i, W_ij

In [224]:
ordered_word_pair('ab','aa')

('aa', 'ab')

In [225]:
ordered_word_pair('aa','ab')

('aa', 'ab')

In [201]:
def compute_pmi(tokenized_data_idx, window_size=10, threshold=0.1, min_count=2, normalize=True):

    W = 0 #compte des fenêtres

    W_ij = dict()
    W_i = dict()

    #tokenized_data = liste des mots pour chaque doc
    for words in tqdm(tokenized_data_idx):
        #Pour chaque document, calcul de toutes les fréquences Wi Wij
        #words = liste des indices des mots de 1 doc

        #Initialisation de la fenêtre
        q = []
        for i in range(min(window_size, len(words))):
            #On ajoute à la queue les indices des mots de la fenetre
            #en partant du début du doc
            q.append(words[i])
        #Update du nb de fenetres
        W+=1
        #Update de W(i) et W(i,j)
        W_i, W_ij = update_word_and_word_pair_occurrence(q, W_i, W_ij)

        next_word_idx = window_size
        #Tant qu'on a pas fini le doc, on décale la fenêtre
        while next_word_idx<len(words):
            #On décale la fenêtre
            #En enlevant le premier mot
            q.pop(0)
            #Et en ajoutant le mot suivant (son indice)
            q.append(words[next_word_idx])
            next_word_idx+=1
            W+=1
            W_i, W_ij = update_word_and_word_pair_occurrence(q, W_i, W_ij)  
    
    PMI = dict()
    #Pour chaque mot qui apparaissent ensemble
    for (i,j), Wij in tqdm(W_ij.items()):
        #On ne garde pas les couple qui apparaissent trop peu
        if Wij < min_count:
            continue
        #word freq
        Wi = W_i[i]
        Wj = W_i[j]
        pij = Wij/W
        pi = Wi/W
        pj = Wj/W

        pmi=math.log(pij/(pi*pj))

        #Normalisation du PMI
        if normalize :
            pmi = pmi/(-math.log(pij))
        if pmi > threshold:
            PMI[i,j]=pmi
    return PMI

In [236]:
PMI = compute_pmi(tokenized_data, window_size=10, threshold=0.2, min_count=20, normalize=True)

100%|████████████████████████████| 1047689/1047689 [00:00<00:00, 3097811.51it/s]


In [237]:
PMI

{('board', 'split'): 0.3261203298119208,
 ('split', 'stock'): 0.4960920223133638,
 ('champion', 'product'): 0.3453753118639079,
 ('board', 'director'): 0.4357378051086698,
 ('approved', 'director'): 0.3021497070589646,
 ('approved', 'board'): 0.38337732604235364,
 ('approved', 'split'): 0.3199310817338034,
 ('split', 'two'): 0.40903076440357594,
 ('one', 'two'): 0.23255896747579471,
 ('one', 'split'): 0.36486616889995965,
 ('common', 'split'): 0.25569068794520533,
 ('common', 'stock'): 0.5159925872281357,
 ('common', 'share'): 0.4258057180972178,
 ('share', 'stock'): 0.24328747617812677,
 ('shareholder', 'split'): 0.24795314285309747,
 ('approved', 'shareholder'): 0.376146315287115,
 ('record', 'split'): 0.3448556324960068,
 ('record', 'shareholder'): 0.2725860684210389,
 ('april', 'record'): 0.5771435489583681,
 ('april', 'split'): 0.2772833780497332,
 ('april', 'shareholder'): 0.21605622200923666,
 ('board', 'voted'): 0.35415270331874893,
 ('board', 'recommend'): 0.33029873789919384,

# test pmi importé de la librairie convectors (pour tester seulement et comparer ? a virer apres)

In [238]:
from convectors.linguistics import pmi

In [239]:
_pmi = pmi(tokenized_data,
               normalize=True,
               min_count=2,
               window_size=10,
               undirected=True,
               minimum=0.1)

In [240]:
print(len(PMI))
print(len(_pmi))

30524
23969


# 4 Construction du graphe <a name="tfidf"></a>

[Retour table des matières](#up)

Nous allons maintenant construire le graphe.

Pour cela, nous commençons par créer un sommet par mot du vocabulaire, et une arête par couple de mots (qui ont un score PMI supérieur à un certain seuil) avec comme poids le score PMI.

Ensuite nous créons une arête par couple (document, mot

In [207]:
def create_graph(PMI, TFIDF, vocab):
    nodes = set()
    edges = []
    
    #aretes entre les mots
    #list_pmi = list(PMI.items())
    for (u,v), w in PMI.items():
        mot_u = u #vocab[u] #REPLACE here
        mot_v = v #vocab[v]
        edges.append((mot_u,mot_v,w))
        nodes.add(mot_u)
        nodes.add(mot_v)
    
    l = len(edges)
        
    #aretes entre mots et doc
    rows, cols = TFIDF.nonzero()
    for r, c in zip(rows, cols):
        #r = doc car ils sont sur les lignes
        #c = mot car sur les colonnes
        mot = vocab[c]
        if mot not in nodes:
            continue
        triplet = (r, mot, TFIDF[r,c])
        edges.append(triplet)  
        
    l1 = len(edges)
    #construction du graphe
    G = nx.Graph()
    G.add_nodes_from(range(TFIDF.shape[0]))
    G.add_weighted_edges_from(edges)

    #Ajout des boucles
    for node in G.nodes:
        G.add_edge(node, node, weight=1)
    
    return G

In [208]:
def build_graph(data,
                no_below=1,
                no_above=0.2,
                max_words=100000,
                window_size=15,
                pmi_threshold=0.3,
                min_count=21,
                normalize=True):
    
    word_tok, dictionary = prep(data, no_below, no_above)
    
    TFIDF, tokenized_data, tokenized_data_idx, vocab = compute_tfidf(data, dictionary, word_tok,0)
    
    n_docs, n_words = TFIDF.shape
    
    #REPLACE here tokenized_data_idx with tokenized_data
    PMI = compute_pmi(tokenized_data, window_size, pmi_threshold, min_count, normalize)

    G = create_graph(PMI, TFIDF, vocab)
    
    return G

In [209]:
G = build_graph(doc_content_list)

100%|████████████████████████████| 1346371/1346371 [00:00<00:00, 3207212.21it/s]


In [210]:
print(G.number_of_nodes())
print(G.number_of_edges())

14227
338560


In [211]:
TFIDF[0]

<1x7101 sparse matrix of type '<class 'numpy.float64'>'
	with 26 stored elements in Compressed Sparse Row format>

In [212]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
   'This is the first document.',
   'This document is the second document.',
  'And this is the third one.',
 'Is this the first document?']
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)


In [213]:
vectorizer.get_feature_names_out()



array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
       'this'], dtype=object)

In [214]:
print(X.shape)

(4, 9)


In [215]:
print(X)

  (0, 1)	0.46979138557992045
  (0, 2)	0.5802858236844359
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 8)	0.38408524091481483
  (1, 5)	0.5386476208856763
  (1, 1)	0.6876235979836938
  (1, 6)	0.281088674033753
  (1, 3)	0.281088674033753
  (1, 8)	0.281088674033753
  (2, 4)	0.511848512707169
  (2, 7)	0.511848512707169
  (2, 0)	0.511848512707169
  (2, 6)	0.267103787642168
  (2, 3)	0.267103787642168
  (2, 8)	0.267103787642168
  (3, 1)	0.46979138557992045
  (3, 2)	0.5802858236844359
  (3, 6)	0.38408524091481483
  (3, 3)	0.38408524091481483
  (3, 8)	0.38408524091481483


In [216]:
for idx in range(TFIDF.shape[1]):
    if TFIDF[0,idx] != 0.0:
        print (vocab[idx])

also
annual
approved
approves
april
authorized
board
capital
champion
common
company
director
fiv
inc
increase
meeting
one
product
recommend
record
share
shareholder
split
stock
two
voted


In [221]:
PMI[.keys()]

dict_keys([('board', 'split'), ('split', 'stock'), ('champion', 'product'), ('board', 'director'), ('approved', 'director'), ('approved', 'board'), ('approved', 'split'), ('split', 'two'), ('one', 'two'), ('one', 'split'), ('common', 'split'), ('common', 'stock'), ('common', 'share'), ('share', 'stock'), ('shareholder', 'split'), ('approved', 'shareholder'), ('record', 'split'), ('record', 'shareholder'), ('april', 'record'), ('april', 'split'), ('april', 'shareholder'), ('board', 'voted'), ('board', 'recommend'), ('recommend', 'shareholder'), ('annual', 'recommend'), ('annual', 'shareholder'), ('annual', 'voted'), ('board', 'meeting'), ('meeting', 'recommend'), ('meeting', 'shareholder'), ('meeting', 'voted'), ('annual', 'meeting'), ('authorized', 'board'), ('authorized', 'shareholder'), ('annual', 'authorized'), ('authorized', 'increase'), ('authorized', 'meeting'), ('authorized', 'stock'), ('authorized', 'share'), ('april', 'reuter'), ('inc', 'system'), ('complete', 'inc'), ('comput

In [218]:
TFIDF.shape

(7674, 7101)

In [219]:
print(G.number_of_nodes())
print(G.number_of_edges())

14227
338560


In [220]:
G.nodes

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 

In [191]:
len(dictionary)

7088

In [192]:
TFIDF.shape[0]+TFIDF.shape[1]

14775

In [193]:
#nx.draw(G)
print(G.number_of_nodes())
print(G.number_of_edges())

13240
328741


In [194]:
#Test avec librairie textgraph

G2 = TextGraph().fit_transform(doc_content_list)
commu2 = louvain(G2)

print(G2.number_of_nodes())
print(G2.number_of_edges())

print(len(set(commu2.values())))

NameError: name 'louvain' is not defined

In [ ]:
vocab[5678]

In [ ]:
dictionary[5678]

In [ ]:
for i in G.neighbors(0):
    print(i)

In [ ]:
tokenized_data[0]